In [1]:
import pandas as pd

# --- STEP 1: Load SEC Submission + Numeric Data ---
sub = pd.read_csv("sub.txt", sep='\t', low_memory=False)
num = pd.read_csv("num.txt", sep='\t', low_memory=False)

# --- STEP 2: Filter for Financial Entities Including Investment Banks ---
relevant_sics = [6111, 6211, 6282, 6719, 6726, 6799]  # Investment banks, brokers, asset managers, holding companies
institutions = sub[sub['sic'].isin(relevant_sics)]

# --- STEP 3: Filter Numeric Values ---
num_filtered = num[num['adsh'].isin(institutions['adsh'])]

tags_needed = [
    'Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax', 'SalesRevenueNet','RetainedEarningsAccumulatedDeficit',
    'AssetsCurrent', 'Assets',
    'LiabilitiesCurrent', 'Liabilities',
    'InterestExpense', 'InterestExpenseOperating', 'InterestAndDebtExpense',
    'LongTermDebt', 'DebtLongtermAndShorttermCombinedAmount',
    'StockholdersEquity', 'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest','OperatingIncomeLoss'
]
num_filtered = num_filtered[num_filtered['tag'].isin(tags_needed)]

# --- STEP 4: Pivot and Merge ---
pivot_df = num_filtered.pivot_table(index='adsh', columns='tag', values='value', aggfunc='first').reset_index()
merged = institutions[['adsh', 'name', 'sic','countryba', 'stprba']]
final_df = pd.merge(merged, pivot_df, on='adsh')
# Rename for clarity
final_df = final_df.rename(columns={
    'name': 'Counterparty_Name',
    'countryba': 'Country',
    'stprba': 'State'
})

In [2]:
# --- STEP 5: Fallback for Missing Financial Fields ---
def safe_combine(df, cols):
    valid_cols = [col for col in cols if col in df.columns]
    if not valid_cols:
        return pd.Series([None] * len(df), index=df.index)
    result = df[valid_cols[0]]
    for col in valid_cols[1:]:
        result = result.combine_first(df[col])
    return result

final_df['Revenue'] = safe_combine(final_df, ['Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax', 'SalesRevenueNet'])
final_df['Debt'] = safe_combine(final_df, ['LongTermDebt', 'DebtLongtermAndShorttermCombinedAmount'])
final_df['Equity'] = safe_combine(final_df, ['StockholdersEquity', 'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest'])
final_df['Interest_Expense'] = safe_combine(final_df, ['InterestExpense', 'InterestExpenseOperating', 'InterestAndDebtExpense'])
final_df['Current_Assets'] = safe_combine(final_df, ['AssetsCurrent', 'Assets'])
final_df['Current_Liabilities'] = safe_combine(final_df, ['LiabilitiesCurrent', 'Liabilities'])
final_df['Total_Assets'] = safe_combine(final_df, ['Assets'])
final_df['Total_Liabilities'] = safe_combine(final_df, ['Liabilities'])
final_df['Retained_Earnings'] = safe_combine(final_df, ['RetainedEarningsAccumulatedDeficit'])
final_df['Operating_Income'] = safe_combine(final_df, ['OperatingIncomeLoss'])


# --- STEP 6: Impute Missing Financial Values ---
financial_cols = ['Revenue', 'Debt', 'Equity', 'Interest_Expense', 'Current_Assets', 'Current_Liabilities','Total_Assets','Total_Liabilities','Retained_Earnings','Operating_Income']



In [3]:
for col in financial_cols:
    final_df[col] = final_df.groupby('sic')[col].transform(lambda x: x.fillna(x.median()))
for col in financial_cols:
    final_df[col] = final_df[col].fillna(final_df[col].median())

final_df = final_df[final_df['Debt'] >= 0]

In [8]:
# --- STEP 7: Add Counterparty ID and Map Sector from SIC ---
final_df['Counterparty_ID'] = ['C' + str(i + 1).zfill(3) for i in range(len(final_df))]

sic_map = {
    6111: 'Credit Agency / Investment Bank',
    6211: 'Broker',
    6282: 'Asset Manager',
    6719: 'Holding Company',
    6726: 'Investment Office',
    6799: 'Investor / PE'
}
final_df['Sector'] = final_df['sic'].map(sic_map)

# --- STEP 8: Impute Sector Based on Name if SIC Wasn't Mapped ---
def infer_sector(name):
    name = str(name).upper()
    if 'BROKER' in name or 'SECURITIES' in name:
        return 'Broker'
    elif 'ASSET' in name or 'INVESTMENT' in name or 'FUND' in name:
        return 'Asset Manager'
    elif 'BANK' in name or 'CAPITAL' in name or 'MORTGAGE' in name or 'CREDIT' in name:
        return 'Credit Agency / Investment Bank'
    elif 'HOLDING' in name or 'HOLDINGS' in name:
        return 'Holding Company'
    elif 'PARTNER' in name or 'PARTNERS' in name or 'EQUITY' in name or 'VENTURE' in name:
        return 'Investor / PE'
    return 'Other'

final_df['Sector'] = final_df.apply(
    lambda row: row['Sector'] if pd.notna(row['Sector']) else infer_sector(row['Counterparty_Name']),
    axis=1
)

final_df['Z1'] = (final_df['Current_Assets'] - final_df['Current_Liabilities']) / final_df['Total_Assets']
final_df['Z2'] = final_df['Retained_Earnings'] / final_df['Total_Assets']
final_df['Z3'] = final_df['Operating_Income'] / final_df['Total_Assets']
final_df['Z4'] = (final_df['Total_Assets'] - final_df['Total_Liabilities']) / final_df['Total_Liabilities']
final_df['Z5'] = final_df['Revenue'] / final_df['Total_Assets']

# --- STEP 9: Final Output ---
result = final_df[['Counterparty_ID', 'Counterparty_Name', 'Revenue', 'Debt', 'Equity',
                   'Interest_Expense', 'Current_Assets', 'Current_Liabilities','Country','State','Sector','Z1', 'Z2', 'Z3', 'Z4', 'Z5']]

# Preview
display(result.head())

,Counterparty_ID,Counterparty_Name,Revenue,Debt,Equity,Interest_Expense,Current_Assets,Current_Liabilities,Country,State,Sector,Z1,Z2,Z3,Z4,Z5
0,C001,FRANKLIN RESOURCES INC,7.390000e+07,9.167300e+09,-4.503000e+08,2.310000e+07,3.246450e+10,1.024270e+10,US,CA,Asset Manager,0.684495,0.367143,0.006361,2.169526,0.002276
1,C002,FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE,2.431590e+08,6.878500e+10,7.768200e+10,9.087400e+10,2.040000e+11,4.255074e+12,US,DC,Credit Agency / Investment Bank,-19.858206,-0.189338,0.000213,-0.952057,0.001192
2,C003,SEI INVESTMENTS CO,0.000000e+00,9.970000e+08,2.131828e+09,4.460000e+08,1.698670e+08,7.485300e+07,US,PA,Broker,0.037704,0.302613,0.218945,4.826677,0.000000
3,C004,SCHWAB CHARLES CORP,4.187000e+09,9.970000e+08,2.000000e+07,7.170000e+08,1.586000e+09,2.304100e+10,US,TX,Broker,-13.527743,21.375158,0.028723,-0.931166,2.639975
4,C005,RAYMOND JAMES FINANCIAL INC,8.870000e+08,9.970000e+08,1.167300e+10,4.460000e+08,2.700000e+07,7.132500e+10,US,FL,Broker,-2640.666667,440.518519,1.687185,-0.999621,32.851852


2. Financial Ratio Analysis

In [10]:
import numpy as np
# Safeguard denominators
final_df["Equity"] = final_df["Equity"].replace(0, np.nan)



final_df.loc[:, "Debt_to_Equity"] = final_df["Debt"] / final_df["Equity"]
final_df.loc[:, "Interest_Coverage"] = final_df["Revenue"] / final_df["Interest_Expense"]
final_df.loc[:, "Current_Ratio"] = final_df["Current_Assets"] / final_df["Current_Liabilities"]



# Define the ratio columns to check
ratio_cols = ["Debt_to_Equity", "Interest_Coverage", "Current_Ratio"]

# Keep only rows where all ratio columns are finite
final_df = final_df[np.isfinite(final_df[ratio_cols]).all(axis=1)]

3. Internal Rating Assignment

In [13]:
def assign_rating(row):
    # --- Distress Override: Negative Equity Scenario ---
    if row["Debt_to_Equity"] < 0:
        return "CCC"  # Firm is technically insolvent

    score = 0

    # --- Debt to Equity Scoring ---
    if row["Debt_to_Equity"] < 1.5:
        score += 2
    elif row["Debt_to_Equity"] < 2.5:
        score += 1

    # --- Interest Coverage Scoring ---
    if row["Interest_Coverage"] > 5:
        score += 2
    elif row["Interest_Coverage"] > 2:
        score += 1

    # --- Current Ratio Scoring ---
    if row["Current_Ratio"] > 1.5:
        score += 2
    elif row["Current_Ratio"] > 1.0:
        score += 1

    # --- Map to Rating Scale ---
    ratings = ["CCC", "B", "BB", "BBB", "A", "AA", "AAA"]
    return ratings[min(score, len(ratings) - 1)]

final_df["Internal_Rating"] = final_df.apply(assign_rating, axis=1)


In [15]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Define possible categorical values
product_types = ['Loan', 'Bond', 'Repo', 'Derivative', 'Credit Card', 'Line of Credit']
collateral_types = ['Gov Bonds', 'Corporate Bonds', 'Cash', 'Real Estate', 'None']
seniority_levels = ['Senior Secured', 'Senior Unsecured', 'Subordinated']

# Simulate additional LGD-relevant variables
n = len(final_df)
final_df['Exposure_Amount'] = np.random.uniform(1e6, 20e6, n).round(2)
final_df['Product_Type'] = np.random.choice(product_types, n)
final_df['Collateral_Type'] = np.random.choice(collateral_types, n)
final_df['Collateral_Value'] = np.random.uniform(0, 20e6, n).round(2)
final_df['Haircut_%'] = np.where(final_df['Collateral_Type'] == 'None', 1.0, np.random.uniform(0.05, 0.4, n).round(2))
final_df['Seniority'] = np.random.choice(seniority_levels, n)
final_df['Recovery_Lag_Months'] = np.random.choice([3, 6, 12, 18], n)



# Calculate Net Collateral Value
final_df['Net_Collateral'] = final_df['Collateral_Value'] * (1 - final_df['Haircut_%'])

# Compute Collateral Coverage Ratio (CCR)
final_df['CCR'] = final_df['Net_Collateral'] / final_df['Exposure_Amount']


# Function to assign base LGD from product type
def base_lgd_from_product(product):
    return {
        'Loan': 0.45,         # Partially collateralized
        'Bond': 0.60,         # Often unsecured or subordinated
        'Repo': 0.08,         # Fully collateralized, low LGD
        'Derivative': 0.15,   # Netting + collateral reduce LGD
        'Credit Card': 0.90,  # Unsecured retail
        'Line of Credit': 0.85  # Unsecured revolving
    }.get(product, 0.50)  # Fallback default


def adjust_lgd_from_collateral(collateral):
    return {
        'Cash': -0.05,
        'Gov Bonds': -0.04,
        'Corporate Bonds': -0.02,
        'Real Estate': 0.00,
        'None': 0.20
    }.get(collateral, 0.00)

# Function to adjust LGD based on seniority
def adjust_lgd_from_seniority(level):
    return {
        'Senior Secured': -0.10,
        'Senior Unsecured': 0.00,
        'Subordinated': 0.10
    }.get(level, 0.00)

# LGD adjustment based on CCR tier
def adjust_lgd_from_ccr(ccr):
    if ccr >= 1.0:
        return -0.15  # over-collateralized
    elif ccr >= 0.75:
        return -0.10
    elif ccr >= 0.5:
        return -0.05
    elif ccr >= 0.25:
        return 0.00
    else:
        return 0.10  # low or no collateral coverage

# Sector-based LGD adjustments
def adjust_lgd_from_sector(sector):
    if "bank" in sector.lower():
        return -0.05
    elif "hedge" in sector.lower():
        return 0.10
    elif "asset manager" in sector.lower():
        return 0.05
    elif "broker" in sector.lower():
        return 0.00
    else:
        return 0.00

# Calculate full institutional LGD using all adjustments
final_df['LGD_Institutional_Enhanced'] = final_df.apply(
    lambda row: min(
        max(
            base_lgd_from_product(row['Product_Type']) +
            adjust_lgd_from_collateral(row['Collateral_Type']) +
            adjust_lgd_from_seniority(row['Seniority']) +
            adjust_lgd_from_ccr(row['CCR']) +
            adjust_lgd_from_sector(row['Sector']),
            0.0
        ),
        1.0
    ),
    axis=1
).round(2)


In [ ]:
final_df

In [18]:
 #Moody's DRD-style PD mapping
rating_pd_map = {
    "AAA": 0.0001, "AA": 0.0002, "A": 0.0005, "BBB": 0.002,
    "BB": 0.01, "B": 0.05, "CCC": 0.20, "CC": 0.30, "C": 0.50, "D": 1.0
}
final_df['Mapped_PD'] = final_df['Internal_Rating'].map(rating_pd_map)

In [20]:
X_counterparty = final_df[['Z1', 'Z2', 'Z3', 'Z4', 'Z5']].replace([np.inf, -np.inf], np.nan).fillna(0)


In [22]:
# Train logistic regression on American Bankruptcy data (latest year)

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score


bankruptcy_df = pd.read_csv("american_bankruptcy.csv")

# Recalculate Z-score variables
bankruptcy_df['Z1'] = (bankruptcy_df['X1'] - bankruptcy_df['X14']) / bankruptcy_df['X10']
bankruptcy_df['Z2'] = bankruptcy_df['X15'] / bankruptcy_df['X10']
bankruptcy_df['Z3'] = bankruptcy_df['X12'] / bankruptcy_df['X10']
bankruptcy_df['Z4'] = (bankruptcy_df['X10'] - bankruptcy_df['X17']) / bankruptcy_df['X17']
bankruptcy_df['Z5'] = bankruptcy_df['X9'] / bankruptcy_df['X10']
bankruptcy_df['Altman_Z'] = (
    1.2 * bankruptcy_df['Z1'] +
    1.4 * bankruptcy_df['Z2'] +
    3.3 * bankruptcy_df['Z3'] +
    0.6 * bankruptcy_df['Z4'] +
    1.0 * bankruptcy_df['Z5']
)

# Ratios for rating
bankruptcy_df['Debt_to_Equity'] = bankruptcy_df['X11'] / bankruptcy_df['X15']
bankruptcy_df['Interest_Coverage'] = bankruptcy_df['X16'] / bankruptcy_df['X13']
bankruptcy_df['Current_Ratio'] = bankruptcy_df['X1'] / bankruptcy_df['X14']

# Clean extreme/inf values
ratio_cols = ['Debt_to_Equity', 'Interest_Coverage', 'Current_Ratio']
bankruptcy_df = bankruptcy_df[np.isfinite(bankruptcy_df[ratio_cols]).all(axis=1)]
bankruptcy_df = bankruptcy_df[bankruptcy_df['Debt_to_Equity'] >= 0]

# Assign internal rating
def assign_internal_rating(row):
    if row['Debt_to_Equity'] < 0:
        return "CCC"
    score = 0
    if row["Debt_to_Equity"] < 1.5:
        score += 2
    elif row["Debt_to_Equity"] < 2.5:
        score += 1
    if row["Interest_Coverage"] > 5:
        score += 2
    elif row["Interest_Coverage"] > 2:
        score += 1
    if row["Current_Ratio"] > 1.5:
        score += 2
    elif row["Current_Ratio"] > 1.0:
        score += 1
    ratings = ["CCC", "B", "BB", "BBB", "A", "AA", "AAA"]
    return ratings[min(score, len(ratings) - 1)]

bankruptcy_df['Internal_Rating'] = bankruptcy_df.apply(assign_internal_rating, axis=1)

# Map Moody’s-style PDs
rating_pd_map = {
    "AAA": 0.0001, "AA": 0.0002, "A": 0.0005, "BBB": 0.002,
    "BB": 0.01, "B": 0.05, "CCC": 0.20, "CC": 0.30, "C": 0.50, "D": 1.0
}
bankruptcy_df['Mapped_PD'] = bankruptcy_df['Internal_Rating'].map(rating_pd_map)

# Keep only latest record per company
latest_panel_df = bankruptcy_df.sort_values("year").drop_duplicates(subset="company_name", keep="last")


# Train logistic regression
features = ['Z1', 'Z2', 'Z3', 'Z4', 'Z5']
X_train = latest_panel_df[features].replace([np.inf, -np.inf], np.nan).fillna(0)
y_train = latest_panel_df['status_label'].apply(lambda x: 1 if x == 'failed' else 0)
log_reg = LogisticRegression(max_iter=1000, class_weight='balanced')
log_reg.fit(X_train, y_train)

# Predict PD for counterparty dataset
final_df['PD_Logistic'] = log_reg.predict_proba(X_counterparty)[:, 1]

# Output preview
display(final_df[['Counterparty_ID', 'Counterparty_Name', 'Internal_Rating', 'Mapped_PD', 'PD_Logistic']].sort_values(by='PD_Logistic', ascending=False).head(10))

,Counterparty_ID,Counterparty_Name,Internal_Rating,Mapped_PD,PD_Logistic
9,C010,OPPENHEIMER HOLDINGS INC,BB,0.0100,0.637765
39,C039,TRILLER GROUP INC.,CCC,0.2000,0.588207
10,C011,GOLDMAN SACHS GROUP INC,CCC,0.2000,0.557811
4,C005,RAYMOND JAMES FINANCIAL INC,BB,0.0100,0.548658
81,C081,"VIRTU FINANCIAL, INC.",BB,0.0100,0.540025
14,C015,MORGAN STANLEY,B,0.0500,0.538643
1,C002,FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE,BB,0.0100,0.538238
88,C088,"APOLLO GLOBAL MANAGEMENT, INC.",A,0.0005,0.538097
22,C022,FEDERAL HOME LOAN MORTGAGE CORP,BB,0.0100,0.537319
52,C052,BLACKSTONE INC.,A,0.0005,0.536410


In [25]:
# --- STEP 1: Calculate Altman Z-Score Variables ---
final_df['Z1'] = (final_df['Current_Assets'] - final_df['Current_Liabilities']) / final_df['Total_Assets']
final_df['Z2'] = final_df['Retained_Earnings'] / final_df['Total_Assets']
final_df['Z3'] = final_df['Operating_Income'] / final_df['Total_Assets']
final_df['Z4'] = (final_df['Total_Assets'] - final_df['Total_Liabilities']) / final_df['Total_Liabilities']
final_df['Z5'] = final_df['Revenue'] / final_df['Total_Assets']

# --- STEP 2: Compute Altman Z-Score ---
final_df['Altman_Z'] = (
    1.2 * final_df['Z1'] +
    1.4 * final_df['Z2'] +
    3.3 * final_df['Z3'] +
    0.6 * final_df['Z4'] +
    1.0 * final_df['Z5']
)

# --- STEP 3: Assign Z-Zone Based on Altman Z-Score ---
def zscore_zone(z):
    if z < 1.8:
        return 'distress'
    elif z <= 3.0:
        return 'grey'
    else:
        return 'safe'

final_df['Z_Zone'] = final_df['Altman_Z'].apply(zscore_zone)

# --- STEP 4: Compute Final Weighted PD ---
def weighted_pd(row):
    if row['Z_Zone'] == 'safe':
        return 0.8 * row['Mapped_PD'] + 0.2 * row['PD_Logistic']
    elif row['Z_Zone'] == 'grey':
        return 0.5 * row['Mapped_PD'] + 0.5 * row['PD_Logistic']
    else:  # distress
        return 0.3 * row['Mapped_PD'] + 0.7 * row['PD_Logistic']

final_df['Final_PD'] = final_df.apply(weighted_pd, axis=1)

# Show preview
final_df[['Counterparty_ID', 'Counterparty_Name', 'Altman_Z', 'Z_Zone', 'Mapped_PD', 'PD_Logistic', 'Final_PD']].sort_values(by='Final_PD', ascending=False).head(10)

,Counterparty_ID,Counterparty_Name,Altman_Z,Z_Zone,Mapped_PD,PD_Logistic,Final_PD
39,C039,TRILLER GROUP INC.,-4.369921,distress,0.20,0.588207,0.471745
10,C011,GOLDMAN SACHS GROUP INC,-678.068327,distress,0.20,0.557811,0.450467
9,C010,OPPENHEIMER HOLDINGS INC,-2579.017325,distress,0.01,0.637765,0.449435
48,C048,FEDERAL HOME LOAN BANK OF SAN FRANCISCO,-0.017571,distress,0.20,0.526945,0.428861
49,C049,FEDERAL HOME LOAN BANK OF TOPEKA,0.126492,distress,0.20,0.525647,0.427953
71,C071,FEDERAL HOME LOAN BANK OF NEW YORK,0.141466,distress,0.20,0.525486,0.427840
8,C009,FEDERAL AGRICULTURAL MORTGAGE CORP,0.246106,distress,0.20,0.524915,0.427440
80,C080,"ROBINHOOD MARKETS, INC.",0.985795,distress,0.20,0.509104,0.416373
14,C015,MORGAN STANLEY,-20.435926,distress,0.05,0.538643,0.392050
4,C005,RAYMOND JAMES FINANCIAL INC,-2514.254284,distress,0.01,0.548658,0.387061


In [ ]:
#final_df.to_csv("df_sample.csv")

In [28]:


# ----------------------
# Step 1: Map Exposure Category
# ----------------------
exposure_type_map = {
    'Loan': 'Term',
    'Bond': 'Term',
    'Repo': 'Other',
    'Derivative': 'Other',
    'Credit Card': 'Revolving',
    'Line of Credit': 'Revolving'
}
final_df['Exposure_Category'] = final_df['Product_Type'].map(exposure_type_map).fillna('Other')

# ----------------------
# Step 2: Assign credit conversion factor and Undrawn Limits for Revolving
# ----------------------
final_df['CCF'] = final_df['Exposure_Category'].map({
    'Revolving': 0.75,
    'Term': 1.0
}).fillna(1.0)

final_df['Undrawn_Limit'] = np.where(
    final_df['Exposure_Category'] == 'Revolving',
    0.25 * final_df['Assets'].fillna(0),
    0
)

# ----------------------
# Step 3: Term Loan Amortized EAD Calculation
# ----------------------
loan_term_months = 60
annual_rate = 0.06
monthly_rate = annual_rate / 12

def monthly_payment(principal, r, n):
    if principal == 0 or r == 0:
        return 0
    return (principal * r * (1 + r)**n) / ((1 + r)**n - 1)

def remaining_principal(p, r, n, ttd):
    if p == 0 or r == 0:
        return 0
    return p * ((1 + r)**n - (1 + r)**ttd) / ((1 + r)**n - 1)

final_df['Exposure_Amount'] = final_df['Exposure_Amount'].fillna(0)
final_df['Monthly_Installment'] = final_df['Exposure_Amount'].apply(
    lambda x: monthly_payment(x, monthly_rate, loan_term_months)
)

np.random.seed(42)
final_df['Time_to_Default'] = final_df['Final_PD'].apply(
    lambda pd: np.random.randint(1, min(loan_term_months, int((1 - pd) * loan_term_months)) + 1)
)

final_df['EAD_Term_Amortized'] = final_df.apply(
    lambda row: remaining_principal(row['Exposure_Amount'], monthly_rate, loan_term_months, row['Time_to_Default'])
    if row['Exposure_Category'] == 'Term' else 0,
    axis=1
)

# ----------------------
# Step 4: Standardized approach for counterparty risk EAD Calculation for Derivatives/Repos
# ----------------------
def calculate_saccr_ead(row):
    alpha = 1.4
    exposure = row['Exposure_Amount'] if not pd.isna(row['Exposure_Amount']) else 0
    collateral = row['Net_Collateral'] if not pd.isna(row['Net_Collateral']) else 0
    haircut = row['Haircut_%'] if not pd.isna(row['Haircut_%']) else 0.10
    rc = max(exposure - collateral, 0)
    pfe = exposure * haircut
    return round(alpha * (rc + pfe), 2)

final_df['EAD_SACCR'] = final_df.apply(
    lambda row: calculate_saccr_ead(row) if row['Exposure_Category'] == 'Other' else 0,
    axis=1
)

# ----------------------
# Step 5: Final EAD Column
# ----------------------
final_df['EAD'] = np.where(
    final_df['Exposure_Category'] == 'Term',
    final_df['EAD_Term_Amortized'],
    np.where(
        final_df['Exposure_Category'] == 'Revolving',
        final_df['Exposure_Amount'] + final_df['CCF'] * (final_df['Undrawn_Limit'] - final_df['Exposure_Amount']),
        final_df['EAD_SACCR']
    )
)


In [31]:
# Set regulatory floor values
pd_floor = 0.0005  # 0.05% minimum PD
lgd_floor = 0.10   # 10% minimum LGD

# Apply the floors
final_df['Final_PD_Floored'] = final_df['Final_PD'].apply(lambda x: max(x, pd_floor))
final_df['LGD_Enhanced_Floored'] = final_df['LGD_Institutional_Enhanced'].apply(lambda x: max(x, lgd_floor))

# Recalculate Expected Loss using floored values
final_df['Expected_Loss_Floored'] = (
    final_df['Final_PD_Floored'] * final_df['LGD_Enhanced_Floored'] * final_df['EAD']
).round(2)


In [36]:
final_df.to_csv("df_sample.csv")

#Stress testing

In [39]:
# Step 1: Define regulatory stress scenarios and multipliers

# Regulatory sensitivity multipliers based on Basel/DFAST expectations
# For each 1% change in macroeconomic factor, the associated PD/LGD/EAD shift is estimated below
regulatory_multipliers = {
    'GDP': {'PD': 0.10, 'LGD': 0.05, 'EAD': -0.02},         # -1% GDP → +10% PD, +5% LGD, -2% EAD
    'unemployment': {'PD': 0.15, 'LGD': 0.04},              # +1% Unemployment → +15% PD, +4% LGD
    'interest_rate': {'EAD': -0.015},                       # +1% Interest → -1.5% EAD
    'VIX': {'PD': 0.0125}                                   # +1 point VIX → +1.25% PD
}

# Regulatory macroeconomic stress scenarios (illustrative and scaled to plausible Fed/IMF stress)
regulatory_scenarios = {
    'severe': {'GDP': -8.5, 'unemployment': 6.3, 'interest_rate': 3.0, 'VIX': 40},
    'moderate': {'GDP': -3.0, 'unemployment': 2.0, 'interest_rate': 1.0, 'VIX': 20},
    'mild': {'GDP': -1.0, 'unemployment': 1.0, 'interest_rate': 0.5, 'VIX': 10}
}

In [41]:
# Step 2: Stress application function
def apply_regulatory_stress(row, scenario, multipliers):
    pd_adj = row['Final_PD_Floored']
    lgd_adj = row['LGD_Enhanced_Floored']
    ead_adj = row['EAD']
    for macro, shock in scenario.items():
        if 'PD' in multipliers.get(macro, {}):
            pd_adj += shock * multipliers[macro]['PD']
        if 'LGD' in multipliers.get(macro, {}):
            lgd_adj += shock * multipliers[macro]['LGD']
        if 'EAD' in multipliers.get(macro, {}):
            ead_adj *= (1 + shock * multipliers[macro]['EAD'])
    # Ensure values stay within realistic bounds
    return pd.Series([max(pd_adj, 0), min(max(lgd_adj, 0), 1.0), max(ead_adj, 0)])

In [43]:
# Step 3: Apply stress scenarios and compute EL, RWA, breaches
for scenario_name, scenario_values in regulatory_scenarios.items():
    # Apply macro stress scenario to PD, LGD, EAD
    final_df[[f'{scenario_name}_PD', f'{scenario_name}_LGD', f'{scenario_name}_EAD']] = final_df.apply(
        apply_regulatory_stress, axis=1, args=(scenario_values, regulatory_multipliers)
    )
    
    # Compute Expected Loss (EL)
    final_df[f'{scenario_name}_EL'] = (
        final_df[f'{scenario_name}_PD'] *
        final_df[f'{scenario_name}_LGD'] *
        final_df[f'{scenario_name}_EAD']
    )

    # Compute simplified Basel-style risk weights and RWAs
    final_df[f'{scenario_name}_RW'] = final_df[f'{scenario_name}_PD'].apply(lambda pd: min(pd * 12.5, 1.0))
    final_df[f'{scenario_name}_RWA'] = final_df[f'{scenario_name}_RW'] * final_df[f'{scenario_name}_EAD']

In [45]:
# Step 4: Calculate baseline RWA and compare against stressed RWA (severe)
final_df['Baseline_RW'] = final_df['Final_PD_Floored'].apply(lambda pd: min(pd * 12.5, 1.0))
final_df['Baseline_RWA'] = final_df['Baseline_RW'] * final_df['EAD']
final_df['RWA_Shift_Severe'] = final_df['severe_RWA'] - final_df['Baseline_RWA']

# Step 5: Breach detection logic for severe scenario
final_df['RWA_Breach_Severe'] = final_df['RWA_Shift_Severe'] > 20_000_000
final_df['EL_Breach_Severe'] = final_df['severe_EL'] > 10_000_000

# Step 6: Recommend mitigation actions
def mitigation_strategy(row):
    if row['RWA_Breach_Severe'] and row['EL_Breach_Severe']:
        return 'Reduce exposure + Hedge PD risk'
    elif row['RWA_Breach_Severe']:
        return 'Review collateral or tranche down'
    elif row['EL_Breach_Severe']:
        return 'Increase reserves or limit exposure'
    else:
        return 'No action needed'

final_df['Mitigation_Strategy_Severe'] = final_df.apply(mitigation_strategy, axis=1)

In [47]:
final_df.head()

,adsh,Counterparty_Name,sic,Country,State,Assets,AssetsCurrent,DebtLongtermAndShorttermCombinedAmount,InterestAndDebtExpense,InterestExpense,...,mild_EAD,mild_EL,mild_RW,mild_RWA,Baseline_RW,Baseline_RWA,RWA_Shift_Severe,RWA_Breach_Severe,EL_Breach_Severe,Mitigation_Strategy_Severe
0,0000038777-25-000017,FRANKLIN RESOURCES INC,6282.0,US,CA,3.246450e+10,NaN,NaN,NaN,23100000.0,...,6.164323e+09,3.203233e+09,1.0,6.164323e+09,1.0,6.089123e+09,7.145586e+08,True,True,Reduce exposure + Hedge PD risk
1,0000310522-25-000199,FEDERAL NATIONAL MORTGAGE ASSOCIATION FANNIE MAE,6111.0,US,DC,2.040000e+11,NaN,NaN,NaN,NaN,...,2.491414e+07,1.243938e+06,1.0,2.491414e+07,1.0,2.461020e+07,2.888007e+06,False,False,No action needed
2,0000350894-25-000028,SEI INVESTMENTS CO,6211.0,US,PA,2.520003e+09,169867000.0,NaN,NaN,NaN,...,1.867969e+07,1.105161e+06,1.0,1.867969e+07,1.0,1.845181e+07,2.165320e+06,False,False,No action needed
3,0000316709-25-000010,SCHWAB CHARLES CORP,6211.0,US,TX,1.586000e+09,NaN,NaN,NaN,NaN,...,3.937654e+06,3.996746e+05,1.0,3.937654e+06,1.0,3.889617e+06,4.564466e+05,False,False,No action needed
4,0000720005-25-000025,RAYMOND JAMES FINANCIAL INC,6211.0,US,FL,2.700000e+07,NaN,NaN,NaN,NaN,...,2.191269e+06,1.108463e+05,1.0,2.191269e+06,1.0,2.164537e+06,2.540085e+05,False,False,No action needed
